## Goal : 

Create a "DASHBOARD" for predicting electricity costs in any given country based on the energy source i.e coal, oil, solar, wind, or nuclear. 

## Data Collection
> World Energy Consumption, [Visit Kaggle](https://www.kaggle.com/datasets/pralabhpoudel/world-energy-consumption)
> Humand Development Index Table, [Visit Site](https://www.theglobaleconomy.com/rankings/human_development/)

## Data Classification
> Countries in the dataset will be classified as:
>* "Developed" : HDI $\geq$ 0.8 & gdp_per_capita $\geq$ $18,750
>* "Developing" : Otherwise

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import country_converter as coc

In [2]:
world_data = pd.read_csv('https://raw.githubusercontent.com/owid/energy-data/master/owid-energy-data.csv')

In [3]:
world_data.head(15)

,country,year,iso_code,population,gdp,biofuel_cons_change_pct,biofuel_cons_change_twh,biofuel_cons_per_capita,biofuel_consumption,biofuel_elec_per_capita,...,solar_share_elec,solar_share_energy,wind_cons_change_pct,wind_cons_change_twh,wind_consumption,wind_elec_per_capita,wind_electricity,wind_energy_per_capita,wind_share_elec,wind_share_energy
0,Afghanistan,1900,AFG,4832414.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1901,AFG,4879685.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,1902,AFG,4935122.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,1903,AFG,4998861.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,1904,AFG,5063419.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Afghanistan,1905,AFG,5128808.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Afghanistan,1906,AFG,5195038.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Afghanistan,1907,AFG,5262120.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Afghanistan,1908,AFG,5330065.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Afghanistan,1909,AFG,5467828.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Load Webscraped Data for the HDI values for countries

In [4]:
import requests
from bs4 import BeautifulSoup

In [5]:
url = "https://www.theglobaleconomy.com/rankings/human_development/"
page = requests.get(url)

# Webscrape
soup = BeautifulSoup(page.content, 'html.parser')
table = soup.find("table", attrs={"id": "benchmarkTable"})
hdi_table = pd.read_html(str(table))[0].drop('Available data', axis=1)

hdi_table.columns = ["country", "hdi_2021", "hdi_rank_2021"]

In [6]:
hdi_table.head(20)

,country,hdi_2021,hdi_rank_2021
0,Switzerland,0.962,1
1,Norway,0.961,2
2,Iceland,0.959,3
3,Hong Kong,0.952,4
4,Australia,0.951,5
5,Denmark,0.948,6
6,Sweden,0.947,7
7,Ireland,0.945,8
8,Germany,0.942,9
9,Netherlands,0.941,10


## Data Wrangling

In [7]:
# replace some country names to match with country names in the world_data
hdi_table['country'] = hdi_table['country'].replace(['USA', 'R. of Congo', 'UK', 'UAE'], 
                                                    ['United States', 'Congo', 'United Kingdom', 'United Arab Emirates'])

In [8]:
# inner merge two datatable on the country column
world_data = pd.merge(world_data, hdi_table, on = "country")

In [9]:
# create new column gdp_per_capita
world_data["gdp_per_capita"] = round(world_data["gdp"] / world_data["population"], 2)

# create new column status based on predetermined conditions
world_data["status"] = ["developed" if x >= 18750 and y >= 0.8 else "developing" for x,y in zip(world_data["gdp_per_capita"], world_data["hdi_2021"])]

# create new column continent based on coutry's ISO code
world_data["continent"] = coc.convert(names = world_data["iso_code"].tolist(), to = "continent", src = "ISO3")

In [10]:
# reorder the columns 
columns_order = ['country', 'year', 'iso_code', 'continent', 'status', 'population', 'hdi_rank_2021', 'hdi_2021', 'gdp','gdp_per_capita'] + [col for col in world_data.columns if col not in ['country', 'year', 'iso_code', 'continent','status', 'population', 'hdi_rank_2021', 'hdi_2021', 'gdp', 'gdp_per_capita']]

world_data = world_data[columns_order]

In [11]:
world_data

,country,year,iso_code,continent,status,population,hdi_rank_2021,hdi_2021,gdp,gdp_per_capita,...,solar_share_elec,solar_share_energy,wind_cons_change_pct,wind_cons_change_twh,wind_consumption,wind_elec_per_capita,wind_electricity,wind_energy_per_capita,wind_share_elec,wind_share_energy
0,Afghanistan,1900,AFG,Asia,developing,4832414.0,173,0.478,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1901,AFG,Asia,developing,4879685.0,173,0.478,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,1902,AFG,Asia,developing,4935122.0,173,0.478,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,1903,AFG,Asia,developing,4998861.0,173,0.478,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,1904,AFG,Asia,developing,5063419.0,173,0.478,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13510,Zimbabwe,2017,ZWE,Africa,developing,14751101.0,141,0.593,2.194784e+10,1487.88,...,0.136,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN
13511,Zimbabwe,2018,ZWE,Africa,developing,15052191.0,141,0.593,2.271535e+10,1509.11,...,0.218,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN
13512,Zimbabwe,2019,ZWE,Africa,developing,15354606.0,141,0.593,NaN,NaN,...,0.364,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN
13513,Zimbabwe,2020,ZWE,Africa,developing,15669663.0,141,0.593,NaN,NaN,...,0.395,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN


move this dataframe to aws or something so it doesnt have to reload the data everytime

In [ ]:
solar_data = world_data[""]